In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urlparse, parse_qs
import re
import pandas as pd
import numpy as np
import time

### Delhi - Mumbai (Weekdays)

In [1]:
urls = [
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2025-02-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2025-02-05&sort=BEST&travelPurpose=leisure",
]

# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


NameError: name 'webdriver' is not defined

In [ ]:
df1 = pd.DataFrame(all_data)
df1

### Delhi - Mumbai (Weekends)

In [ ]:
urls = [
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL&to=BOM&depart=2025-02-09&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL-BOM/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL&to=BOM&depart=2025-02-09&sort=BEST&travelPurpose=leisure",
]

# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df2 = pd.DataFrame(all_data)
df2

### Bangalore - Kolkata (Weekdays)

In [ ]:
urls = [
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-02-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-02-05&sort=BEST&travelPurpose=leisure"
]

# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df3 = pd.DataFrame(all_data)
df3

### Bangalore - Kolkata (Weekends)

In [ ]:
urls = [
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-02-09&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BLR.AIRPORT-CCU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BLR.AIRPORT&to=CCU.AIRPORT&depart=2025-02-09&sort=BEST&travelPurpose=leisure"
]

# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df4 = pd.DataFrame(all_data)
df4

### Chennai - Hyderabad (Weekdays)

In [ ]:
urls = [
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-02-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-02-05&sort=BEST&travelPurpose=leisure"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df5 = pd.DataFrame(all_data)
df5

### Chennai - Hyderabad (Weekends)

In [ ]:
urls = [
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-02-09&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/MAA.AIRPORT-HYD.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=MAA.AIRPORT&to=HYD.AIRPORT&depart=2025-02-09&sort=BEST&travelPurpose=leisure"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df6 = pd.DataFrame(all_data)
df6

### Mumbai - Goa (Weekdays)

In [ ]:
urls = [
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-02-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-02-05&sort=BEST&travelPurpose=leisure"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df7 = pd.DataFrame(all_data)
df7

### Mumbai - Goa (Weekends)

In [ ]:
urls = [
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-02-09&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/BOM.AIRPORT-GOX.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=BOM.AIRPORT&to=GOX.AIRPORT&depart=2025-02-09&sort=BEST&travelPurpose=leisure"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df8 = pd.DataFrame(all_data)
df8

### Kolkata - Guwahathi (Weekdays)

In [ ]:
urls = [
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-02-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-02-05&sort=BEST&travelPurpose=leisure"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df9 = pd.DataFrame(all_data)
df9

### Kolkata - Guwahathi (Weekends)

In [ ]:
urls = [
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-02-09&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/CCU.AIRPORT-GAU.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=CCU.AIRPORT&to=GAU.AIRPORT&depart=2025-02-09&sort=BEST&travelPurpose=leisure"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df10 = pd.DataFrame(all_data)
df10

### Delhi - Bangalore (Weekdays)

In [ ]:
urls = [
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-18&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-19&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-07&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMIC&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-02-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-02-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMIC&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-02-05&sort=BEST&travelPurpose=leisure"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df11 = pd.DataFrame(all_data)
df11

### Delhi - Bangalore (Weekends)

In [ ]:
urls = [
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-21&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2024-12-22&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-04&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMY&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-01-05&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMIC&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-02-08&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=BUSINESS&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-02-09&sort=BEST&travelPurpose=leisure",
    "https://flights.booking.com/flights/DEL.AIRPORT-BLR.AIRPORT/?type=ONEWAY&adults=1&cabinClass=ECONOMIC&children=&from=DEL.AIRPORT&to=BLR.AIRPORT&depart=2025-02-09&sort=BEST&travelPurpose=leisure"
]


# Airline names list
emp_list = ["IndiGo", "Air India", "Air India Express", "Etihad Airways", "SpiceJet", "Akasa Air"]

# Initialize browser
browser = webdriver.Chrome()

# Data collection list
all_data = []

for url in urls:
    browser.get(url)

    # Extract cabin class from URL
    parsed_url = urlparse(url)
    cabin_class = parse_qs(parsed_url.query).get("cabinClass", ["Not Found"])[0].replace("_", " ").title()
    
    # Handle lazy-loading if needed
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Adjust sleep time if needed
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Extract Prices
    prices = [price.text.strip() for price in soup.find_all('div', class_="FlightCardPrice-module__priceContainer___nXXv2")]

    # Extract Airline Names
    airlines = [airline.text.strip() for airline in soup.find_all('div', class_="Text-module__root--variant-small_1___An5P8") 
                if airline.text.strip() in ["IndiGo", "Air India", "SpiceJet", "Akasa Air", "Vistara"]]

    # Extract Stops
    stops_raw = [stop.text.strip() for stop in soup.find_all('span', class_="Badge-module__text___AGLG9")]
    stops = [stop for stop in stops_raw if "stop" in stop.lower() or "direct" in stop.lower()]

    # Extract Travel Dates
    travel_dates = [date.text.strip() for date in soup.find_all('div', {"data-testid": "flight_card_segment_departure_date_0"})]

    # Extract Departure Cities
    departure_cities = [dep.text.strip() for dep in soup.find_all('div', {"data-testid": "flight_card_segment_departure_airport_0"})]

    # Extract Arrival Cities
    arrival_cities = [arr.text.strip() for arr in soup.find_all('div', {"data-testid": "flight_card_segment_destination_airport_0"})]

    # Extract Durations
    durations = [duration.text.strip() for duration in soup.find_all('div', {"data-testid": "flight_card_segment_duration_0"})]

    # Combine Data
    num_rows = min(len(prices), len(airlines), len(stops), len(travel_dates), len(departure_cities), len(arrival_cities), len(durations))
    for i in range(num_rows):
        all_data.append({
            "Price": prices[i],
            "Airline": airlines[i],
            "Departure City": departure_cities[i],
            "Arrival City": arrival_cities[i],
            "Travel Date": travel_dates[i],
            "Duration": durations[i],
            "Stops": stops[i],
            "Cabin Class": cabin_class  # Added cabin class
        })

    print(f"Processed {url}: {len(all_data)} total rows collected so far")

# Close browser
browser.quit()


In [ ]:
df12 = pd.DataFrame(all_data)
df12